<a href="https://colab.research.google.com/github/QuinnMcGill/makeyourdatafair/blob/feature%2Fcallum-suppression/adult_reweighing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reweighing Technique for Census Income Dataset
This notebook was adapted from AIF360's example https://github.com/Trusted-AI/AIF360/blob/main/examples/demo_reweighing_preproc.ipynb

### Part 1: Install Dependencies

In [2]:
!pip install gower 'aif360[all]'

You can skip this next cell. It is mainly used for development and deletes all the notebook's variables.

In [3]:
!reset

Import packages and define functions to evaluate performance and indvidual fairness. Before you run the next cell, you'll need to put 'adult.data', 'adult.names' and 'adult.test' in '/usr/local/lib/python3.11/dist-packages/aif360/data/raw/adult'

Define performance evaluation helper function.

In [4]:
from IPython.display import Markdown, display
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, classification_report, recall_score, f1_score

from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult

from group_fairness import eval_group_fairness

def eval_performance(y_test, y_pred):
    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Display metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print("\nClassification Report:\n", classification_report(y_test, y_pred))


Define individual fairness evaluation function.

In [5]:
from sklearn.neighbors import NearestNeighbors
import gower

def eval_ind_fairness(x_train, y_train, x_test, y_pred):
  # Compute Gower distance matrix for test samples w.r.t training data
  gower_distances = gower.gower_matrix(x_test, x_train)  # Shape: (num_test_samples, num_train_samples)

  # Find k nearest neighbors (excluding self)
  k = 5  # Adjust as needed
  neighbors = np.argsort(gower_distances, axis=1)[:, 1:k+1]  # Get indices of k nearest neighbors

  # Compute consistency score: Fraction of nearest neighbors with same prediction
  consistencies = []
  for i, neigh_indices in enumerate(neighbors):
      neighbor_preds = y_train[neigh_indices]  # Get predictions of k neighbors from training labels
      consistency = np.mean(neighbor_preds == y_pred[i])  # Fraction with same prediction
      consistencies.append(consistency)

  # Calculate overall consistency score
  individual_fairness_score = np.mean(consistencies)
  return individual_fairness_score

Define disparate impact evaluation function.

In [6]:
import pandas as pd

def eval_disparate_impact(x_test_priv_col, y_pred, unpriv_group_val=0, favourable_outcome=1):
  df = pd.DataFrame({
      'privileged_attribute': x_test_priv_col,
      'prediction': y_pred
  })

  # Calculate selection rates for protected and unprotected groups
  unpriveleged_group = df[df['privileged_attribute'] == unpriv_group_val]
  priveleged_group = df[df['privileged_attribute'] != unpriv_group_val]

  unpriveleged_selection_rate = np.mean(unpriveleged_group['prediction'] == favourable_outcome)
  priveleged_selection_rate = np.mean(priveleged_group['prediction'] == favourable_outcome)

  # Calculate disparate impact
  if priveleged_selection_rate == 0:
      disparate_impact = float('inf') if unpriveleged_selection_rate > 0 else 1.0
  else:
      disparate_impact = unpriveleged_selection_rate / priveleged_selection_rate

  return disparate_impact

Define counterfactual fairness evaluation function.

In [7]:
def evaluate_counterfactual_fairness_sex(model, X):
    """
    Evaluates counterfactual fairness by flipping the 'sex' attribute.
    """
    # Store the original predictions on the test set
    original_predictions = model.predict(StandardScaler().fit_transform(X))

    # Create a copy of the dataset
    X_counterfactual = X.copy()

    # Flip 'sex' (0 -> 1, 1 -> 0)
    if 'sex' in X.columns:
        X_counterfactual['sex'] = 1 - X_counterfactual['sex']
    else:
        raise KeyError(f"The sensitive attribute 'sex' is not present in the dataset.")

    # Get counterfactual predictions
    counterfactual_predictions = model.predict(StandardScaler().fit_transform(X_counterfactual))

    # Compare the original and counterfactual predictions
    comparison = pd.DataFrame({
        'original': original_predictions,
        'counterfactual': counterfactual_predictions,
        'same_decision': original_predictions == counterfactual_predictions
    })

    # Return the comparison dataframe
    return comparison

### Part 2: Load the Data

In [ ]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = load_preproc_data_adult(['sex'])
# Convert the dataset to a DataFrame
df = pd.DataFrame(columns=dataset_orig.feature_names, data=dataset_orig.features)
df['Income Binary'] = dataset_orig.labels  # AIF360 uses labels, rename to match your other code

# Include protected attribute 'sex' explicitly
df['sex'] = dataset_orig.protected_attributes[:, 0]  # First (and only) protected attr: 'sex'

np.random.seed(1)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/aif360/algorithms/preprocessing/optim_preproc_helpers/data_preproc_functions.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sex'] = df['sex'].replace({'Female': 0.0, 'Male': 1.0})


Split the data into training and test sets.

In [9]:
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.8], shuffle=True)

### Part 3: Train and Evaluate the Baseline

In [10]:
scale_orig = StandardScaler()
x_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
w_train = dataset_orig_train.instance_weights.ravel()

lmod = LogisticRegression()
lmod.fit(x_train,
         y_train,
         sample_weight=dataset_orig_train.instance_weights)

LogisticRegression()

In [11]:
x_test = scale_orig.fit_transform(dataset_orig_test.features)
y_test = dataset_orig_test.labels.ravel()

y_pred = lmod.predict(x_test)

In [12]:
# Evaluate performance of baseline model
eval_performance(y_test, y_pred)

Accuracy: 0.8071
Precision: 0.6592
Recall: 0.3934
F1 Score: 0.4927

Classification Report:
               precision    recall  f1-score   support

         0.0       0.83      0.94      0.88      7443
         1.0       0.66      0.39      0.49      2326

    accuracy                           0.81      9769
   macro avg       0.75      0.66      0.69      9769
weighted avg       0.79      0.81      0.79      9769



In [13]:
x_test_df = dataset_orig_test.convert_to_dataframe()[0]
disparate_impact = eval_disparate_impact(x_test_df['sex'], y_pred)
print(f'Disparate Impact Score: {disparate_impact:.4f}')

Disparate Impact Score: 0.0000


In [14]:
individual_fairness_score = eval_ind_fairness(x_train, y_train, x_test, y_pred)
print(f'Individual Fairness Consistency Score: {individual_fairness_score:.4f}')

Individual Fairness Consistency Score: 0.7879


In [15]:
counterfactual_fairness = evaluate_counterfactual_fairness_sex(lmod, x_test_df.drop('Income Binary', axis=1))
fairness_metric_sex = counterfactual_fairness['same_decision'].mean()
print(f'Counterfactual Fairness Score: {fairness_metric_sex:.4f}')

Counterfactual Fairness Score: 0.8091


In [ ]:
# Evaluate Group Fairness on Baseline
orig_model_group_fairness = eval_group_fairness(x_test_df.drop('Income Binary', axis=1), target='Income Binary', protected_attr='sex', mode='model', y_pred=y_pred)
print("\nModel Group Fairness Metrics (Baseline):")
for metric, value in orig_model_group_fairness.items():
    print(f"{metric}: {value:.4f}")



Model Group Fairness Metrics (Baseline):
Statistical Parity Difference: -0.2142
Disparate Impact: 0.0000
Demographic Parity: -0.2142


### Part 4: Train and Evaluate the Model Trained on Reweighted Dataset

Run the reweighing algorithm.

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_rw_train = RW.transform(dataset_orig_train)

In [ ]:
scale_orig = StandardScaler()
x_train_rw = scale_orig.fit_transform(dataset_rw_train.features)
y_train_rw = dataset_rw_train.labels.ravel()
w_train_rw = dataset_rw_train.instance_weights.ravel()

lmod_rw = LogisticRegression()
lmod_rw.fit(x_train_rw,
            y_train_rw,
            sample_weight=dataset_rw_train.instance_weights)

LogisticRegression()

In [ ]:
x_test = scale_orig.fit_transform(dataset_orig_test.features)
y_test = dataset_orig_test.labels.ravel()

y_pred_rw = lmod_rw.predict(x_test)

In [ ]:
# Evaluate performance of model trained on reweighted dataset
eval_performance(y_test, y_pred_rw)

Accuracy: 0.7912
Precision: 0.5800
Recall: 0.4458
F1 Score: 0.5041

Classification Report:
               precision    recall  f1-score   support

         0.0       0.84      0.90      0.87      7443
         1.0       0.58      0.45      0.50      2326

    accuracy                           0.79      9769
   macro avg       0.71      0.67      0.69      9769
weighted avg       0.78      0.79      0.78      9769



In [ ]:
x_test_df = dataset_orig_test.convert_to_dataframe()[0]
disparate_impact_rw = eval_disparate_impact(x_test_df['sex'], y_pred_rw)
print(f'Disparate Impact Score: {disparate_impact_rw:.4f}')

Disparate Impact Score: 0.6646


In [ ]:
individual_fairness_score_rw = eval_ind_fairness(x_train_rw, y_train_rw, x_test, y_pred_rw)
print(f'Individual Fairness Consistency Score: {individual_fairness_score_rw:.4f}')

Individual Fairness Consistency Score: 0.7827


In [ ]:
counterfactual_fairness_rw = evaluate_counterfactual_fairness_sex(lmod_rw, x_test_df.drop('Income Binary', axis=1))
fairness_metric_sex_rw = counterfactual_fairness_rw['same_decision'].mean()
print(f'Counterfactual Fairness Score: {fairness_metric_sex_rw:.4f}')

Counterfactual Fairness Score: 1.0000


In [20]:
# Evaluate Group Fairness on Reweighted Model
after_model_group_fairness = eval_group_fairness(x_test_df.drop('Income Binary', axis=1), target='Income Binary', protected_attr='sex', mode='model', y_pred=y_pred)
print("\nModel Group Fairness Metrics (After):")
for metric, value in after_model_group_fairness.items():
    print(f"{metric}: {value:.4f}")


Model Group Fairness Metrics (After):
Statistical Parity Difference: -0.2142
Disparate Impact: 0.0000
Demographic Parity: -0.2142


### Part 5: Comments

Comments:
- The evaluation scores in the baseline presented here differ slightly from data_exploration.ipynb. This is probably due to the fact that 'load_preproc_data_adult' modifies the format of the tabular data. For more details you can take a look in the debugger or run:
```
print(dataset_orig_train.feature_names)
```
- **Model trained on reweighted data performs slightly worse:** This is expected. Utility - Fairness tradeoff
-**The change in individual fairness score is negligible:** This is expected. The reweighing strategy is used to address Group Fairness on the level of protected attributes, not individual fairness.
- It looks like the test data chosen in this notebook doesnt have any samples where 'sex'=0 and Income>50k=1 which is why we get a disparate impact of 0 when running the baseline against non-reweighted data.


---


Future Work:
- Implement Group Fairness scoring.
- Implement other fairness scoring.
- Maybe we can look at other protected attributes. In this example I've just chosen 'sex'.